In [1]:
!pip install pyspark

You should consider upgrading via the '/Users/alindsplayground/PycharmProjects/SparkML/.venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
print(os.environ.get("JAVA_HOME"))


/Library/Java/JavaVirtualMachines/temurin-21.jdk/Contents/Home


In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
        .master('local')\
        .appName("regression")\
        .getOrCreate()

24/10/07 00:41:46 WARN Utils: Your hostname, Alinds-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.40.35 instead (on interface en0)
24/10/07 00:41:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/07 00:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
!pwd
df=spark.read.format("csv").load("/Users/alindsplayground/PycharmProjects/SparkML/.venv/data/housing.csv",header=True,inferSchema=True)

/Users/alindsplayground/PycharmProjects/SparkML


In [6]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [7]:
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [8]:
!pip3 install jupyter-tabnine
#from pyspark.sql.functions import mono

You should consider upgrading via the '/Users/alindsplayground/PycharmProjects/SparkML/.venv/bin/python3 -m pip install --upgrade pip' command.


In [9]:
!jupyter nbextension install --py jupyter_tabnine
!jupyter nbextension enable --py jupyter_tabnine


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
        

In [10]:
from pyspark.sql import functions as F

df=df.withColumn("id",F.monotonically_increasing_id())

df=df[['id']+ df.columns[:-1]]
df.show()

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  0|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  1|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  2|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  3|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  4| 

In [11]:
df.select('total_rooms').agg({'total_rooms':'avg'}).show()

+------------------+
|  avg(total_rooms)|
+------------------+
|2635.7630813953488|
+------------------+



In [12]:
df.select(*[F.mean(c) for c in df.columns]).show()

+-------+-------------------+----------------+-----------------------+------------------+-------------------+------------------+-----------------+------------------+-----------------------+--------------------+
|avg(id)|     avg(longitude)|   avg(latitude)|avg(housing_median_age)|  avg(total_rooms)|avg(total_bedrooms)|   avg(population)|  avg(households)|avg(median_income)|avg(median_house_value)|avg(ocean_proximity)|
+-------+-------------------+----------------+-----------------------+------------------+-------------------+------------------+-----------------+------------------+-----------------------+--------------------+
|10319.5|-119.56970445736148|35.6318614341087|     28.639486434108527|2635.7630813953488|  537.8705525375618|1425.4767441860465|499.5396802325581|3.8706710029070246|     206855.81690891474|                NULL|
+-------+-------------------+----------------+-----------------------+------------------+-------------------+------------------+-----------------+----------

In [13]:
df.groupBy('ocean_proximity').agg({col:'avg' for  col in df.columns[3:-1]}).show(5)

+---------------+------------------+------------------+-------------------+------------------+------------------+-----------------------+-----------------------+
|ocean_proximity|   avg(households)|   avg(population)|avg(total_bedrooms)|avg(median_income)|  avg(total_rooms)|avg(median_house_value)|avg(housing_median_age)|
+---------------+------------------+------------------+-------------------+------------------+------------------+-----------------------+-----------------------+
|         ISLAND|             276.6|             668.0|              420.4|2.7444200000000003|            1574.6|               380440.0|                   42.4|
|     NEAR OCEAN|501.24454477050415|1354.0086531226486|  538.6156773211568| 4.005784800601957| 2583.700902934537|     249433.97742663656|     29.347253574115875|
|       NEAR BAY| 488.6161572052402|1230.3174672489083|  514.1828193832599| 4.172884759825336| 2493.589519650655|     259212.31179039303|      37.73013100436681|
|      <1H OCEAN| 517.744964

In [14]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

def square(value):
    return value* value

squared_udf=udf(square,FloatType())

df.withColumn("total_room_square",squared_udf('total_rooms')).show(5)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-----------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|total_room_square|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-----------------+
|  0|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|         774400.0|
|  1|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|        5.03958E7|
|  2|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|        2152089.0|
|  3|  -122.25|   37.85|              52

In [15]:
df.show(5)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  0|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  1|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  2|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  3|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  4| 

In [16]:
train,test= df.randomSplit([.7,.3])
train,test

(DataFrame[id: bigint, longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double, ocean_proximity: string],
 DataFrame[id: bigint, longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double, ocean_proximity: string])

In [20]:
#df.columns
excludeList=['id','median_house_value','ocean_proximity']
numerical_col_list=[x for x in train.columns if not x in excludeList] 
numerical_col_list

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [22]:
from pyspark.ml.feature import Imputer

imputer=Imputer(inputCols=numerical_col_list,outputCols=numerical_col_list)

imputer=imputer.fit(train)

train=imputer.transform(train)
test=imputer.transform(test)

train.show(3)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  0|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  1|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  2|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only s

In [26]:
from pyspark.ml.feature import VectorAssembler
va=VectorAssembler(inputCols=numerical_col_list,outputCol='numerical_vector_feature')

train=va.transform(train)

train.show(3)


+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|numerical_vector_feature|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+
|  0|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|    [-122.23,37.88,41...|
|  1|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|    [-122.22,37.86,21...|
|  2|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|    [-122.24,37.85,52...

In [28]:
test=va.transform(test)

In [27]:
train.select('numerical_vector_feature').take(2)

[Row(numerical_vector_feature=DenseVector([-122.23, 37.88, 41.0, 880.0, 129.0, 322.0, 126.0, 8.3252])),
 Row(numerical_vector_feature=DenseVector([-122.22, 37.86, 21.0, 7099.0, 1106.0, 2401.0, 1138.0, 8.3014]))]

In [29]:
from pyspark.ml.feature import StandardScaler

ss=StandardScaler(inputCol='numerical_vector_feature',outputCol='scaled_numerical_vector_feature',withStd=True,withMean=True)

scalar=ss.fit(train)

train=scalar.transform(train)


AttributeError: 'StandardScalerModel' object has no attribute 'test'

In [30]:
test=scalar.transform(test)

In [31]:
from pyspark.ml.feature import StringIndexer

si=StringIndexer(inputCol='ocean_proximity',outputCol='indexer_ocean_proximity')

si=si.fit(train)
train=si.transform(train)
test=si.transform(test)

train.select('indexer_ocean_proximity').take(2)



[Row(indexer_ocean_proximity=3.0), Row(indexer_ocean_proximity=3.0)]

In [33]:
from pyspark.ml.feature import OneHotEncoder

ohe=OneHotEncoder(inputCol='indexer_ocean_proximity',outputCol='ocean_proximity_one_hot')

ohe=ohe.fit(train)
train=ohe.transform(train)
test=ohe.transform(test)

test.select('ocean_proximity_one_hot').take(2)




[Row(ocean_proximity_one_hot=SparseVector(4, {3: 1.0})),
 Row(ocean_proximity_one_hot=SparseVector(4, {3: 1.0}))]

In [34]:
vectorAssemble=VectorAssembler(inputCols=['scaled_numerical_vector_feature','ocean_proximity_one_hot'],
                               outputCol='final_feature_vector')
test=vectorAssemble.transform(test)


AttributeError: 'VectorAssembler' object has no attribute 'train'

In [35]:
train=vectorAssemble.transform(train)

test.select('final_feature_vector').take(2)

[Row(final_feature_vector=DenseVector([-1.3351, 1.0381, 1.8479, -0.616, -0.7135, -0.7677, -0.7244, 0.9295, 0.0, 0.0, 0.0, 1.0])),
 Row(final_feature_vector=DenseVector([-1.3351, 1.0381, 1.8479, -0.7761, -0.7652, -0.8961, -0.7914, 0.0875, 0.0, 0.0, 0.0, 1.0]))]

In [36]:
from pyspark.ml.regression import LinearRegression

lr=LinearRegression(featuresCol='final_feature_vector',
                    labelCol='median_house_value')



In [37]:
lr=lr.fit(train)
pred_train=lr.transform(train).withColumnRenamed('predection','predecited_house_value')
pred_train.show(5)

24/10/07 02:18:22 WARN Instrumentation: [044f10a9] regParam is zero, which might cause numerical instability and overfitting.
24/10/07 02:18:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/10/07 02:18:22 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/10/07 02:18:23 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+-------------------------------+-----------------------+-----------------------+--------------------+------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|numerical_vector_feature|scaled_numerical_vector_feature|indexer_ocean_proximity|ocean_proximity_one_hot|final_feature_vector|        prediction|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+-------------------------------+-----------------------+-----------------------+--------------------+------------------+
|  0|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BA

In [38]:
pred_test=lr.transform(test).withColumnRenamed('predection','predecited_house_value')
pred_test.show(5)

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+-------------------------------+-----------------------+-----------------------+--------------------+------------------+
| id|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|numerical_vector_feature|scaled_numerical_vector_feature|indexer_ocean_proximity|ocean_proximity_one_hot|final_feature_vector|        prediction|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------------+-------------------------------+-----------------------+-----------------------+--------------------+------------------+
|  3|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BA

In [42]:
actual_and_predicted= pred_train[['prediction','median_house_value']]
actual_and_predicted.show(2)

+-----------------+------------------+
|       prediction|median_house_value|
+-----------------+------------------+
|405923.2932992894|          452600.0|
|426032.3325483982|          358500.0|
+-----------------+------------------+
only showing top 2 rows



In [45]:
actual_and_predicted_rdd=actual_and_predicted.rdd
actual_and_predicted_tuple=actual_and_predicted_rdd.map(tuple)

In [49]:
from pyspark.mllib.evaluation import RegressionMetrics

metrics=RegressionMetrics(actual_and_predicted_tuple)
metrics.meanAbsoluteError,metrics.rootMeanSquaredError,metrics.meanSquaredError,metrics.r2

/Users/alindsplayground/PycharmProjects/SparkML/.venv/lib/python3.9/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


(49912.682984750616, 68451.53563775758, 4685612731.167195, 0.64715158890219)